# Forcasting of wind speed


In [1]:
from IPython.display import clear_output

## Load Data

In [2]:
from src.load_data import read_raw_data, make_ready_data
train_data, test_data, scaler = read_raw_data()
train_data.shape, test_data.shape

((70128, 7, 6), (10872, 7, 6))

### Unified Models specs

Here we use the same specs for all the models

In [3]:
# input and output size are the number of series (cities)
input_size = output_size = 7

# hidden size is number of LSTM units, this means 32 as it's BiLSTM
hidden_size = 16

# Only one layer of LSTM
num_layers = 1

## Using LSTM Baseline
### 1 Hour ahead

In [4]:
# build time series features and labels
from src.data_utils import build_dataloader
hours_ahead = 1 #1 Hour ahead, This is to be changed to (5, 10, 50...etc)
xtrain, xval, ytrain, yval = make_ready_data(train_data, feature='speed',gap=hours_ahead)
xtest, ytest = make_ready_data(test_data, train=False, feature='speed', gap=hours_ahead)
train_iter, val_iter, test_iter, device = build_dataloader(xtrain, xval, xtest, ytrain, yval, ytest)

In [5]:
from src.models import LSTM
from src.run import run_train, validate
from src.run import run_test

# build the model
lstm_baseline = LSTM(output_size, input_size, hidden_size, num_layers)
lstm_baseline = lstm_baseline.to(device)

# train the model
lstm_baseline = run_train(lstm_baseline, train_iter, val_iter, num_epochs=10, features_set=1)

#now we test the model
y_true, y_preds = run_test(lstm_baseline, test_iter, scaler, features_set=1)
clear_output()

In [8]:
# get evaluation metrics on test data
from src.vis import results
results(y_true, y_preds, plots=False)

RMSE:  11.844697
MAE:  8.992798


## Using LSTM with BiLinear Pooling Fusion
### 1 Hour ahead

In [6]:
# build time series features and labels including tempreture time series
xtrain, xval, ytrain, yval = make_ready_data(train_data, feature='speed',gap=hours_ahead)
xtrain_temp, xval_temp, _, _ = make_ready_data(train_data, feature='tempreture', gap=hours_ahead)
xtest, ytest = make_ready_data(test_data, train=False, feature='speed', gap=hours_ahead)
xtest_temp, _ = make_ready_data(test_data, train=False, feature='tempreture', gap=hours_ahead)
train_iter, val_iter, test_iter, device = build_dataloader(xtrain, xval, xtest, 
                                                           ytrain, yval, ytest, 
                                                           xtrain_temp, xval_temp,  
                                                           xtest_temp, add_temp=True)

In [7]:
from src.models import BiLinearPoolingLSTM

lstm_model = BiLinearPoolingLSTM(output_size, input_size, hidden_size, num_layers)
lstm_model = lstm_model.to(device)
lstm_model = run_train(lstm_model, train_iter, val_iter, num_epochs=10)

y_true, y_preds = run_test(lstm_model, test_iter, scaler)
clear_output()

In [8]:
results(y_true, y_preds, plots=False)

RMSE:  10.82287
MAE:  7.305436
